In [69]:
import pandas as pd
import numpy as np

In [70]:
customers = pd.read_csv('data_csv/Customers.csv', encoding='unicode_escape')
data_dictionary = pd.read_csv('data_csv/Data_Dictionary.csv')
exchange_rates = pd.read_csv('data_csv/Exchange_Rates.csv')
products = pd.read_csv('data_csv/Products.csv')
sales =  pd.read_csv('data_csv/Sales.csv')
stores = pd.read_csv('data_csv/Stores.csv')


In [71]:
#preprocessing customer table

#standardizing column_names
def standardize_col_names(df: pd.DataFrame)->pd.DataFrame:
        col_names = {val:val.lower().replace(' ','_').strip() for val in df.columns}
        df.rename(columns=col_names,inplace=True)
        return df


def replace_nan_with_na(row):
    if row['state'] == 'Napoli' and pd.isna(row['state_code']):
        row['state_code'] = 'NA'
    return row

def format_date_column(row,date_col_name):
    if not pd.isna(row[date_col_name]):
        date = row[date_col_name].split('/')
        row[date_col_name] = f"{date[-1]}-{date[0]}-{date[1]}"
    return row
        

#standardizing column names
customers = standardize_col_names(customers)
#imputing null values
customers = customers.apply(replace_nan_with_na, axis=1)
#formating birthday in sql 
customers= customers.apply(lambda row : format_date_column(row,date_col_name='birthday'),axis = 1)


In [72]:
#preprocessing products table

def create_product_id(row):
    index = product_category[(product_category['Subcategory'] == row['Subcategory']) & (product_category['Category'] == row['Category'])]['index']
    return index.item()

product_category = products.iloc[:,[-3,-1]].drop_duplicates().reset_index(drop=True).reset_index()
products['category_id'] = [create_product_id(row[1]) for row in products.iterrows()]
products = products.drop(['Subcategory','CategoryKey','Category','SubcategoryKey'],axis=1)
products = standardize_col_names(products)
products['unit_cost_usd'] = products['unit_cost_usd'].str.replace('$', '').str.replace(',','').str.strip().astype(float)
products['unit_price_usd'] = products['unit_price_usd'].str.replace('$', '').str.replace(',','').str.strip().astype(float)
product_category = standardize_col_names(product_category)






In [73]:
#preprocessing sales table
sales = standardize_col_names(sales)
sales= sales.apply(lambda row : format_date_column(row,date_col_name='order_date'),axis = 1)
sales= sales.apply(lambda row : format_date_column(row,date_col_name='delivery_date'),axis = 1)
sales['delivery_date'] = sales['delivery_date'].apply(lambda x : None if pd.isna(x) else x)


In [74]:
#preprocessing stores table
stores = standardize_col_names(stores)
stores = stores.apply(lambda row : format_date_column(row,date_col_name='open_date'),axis=1)
stores['square_meters'] = stores['square_meters'].fillna(0)

In [75]:
#preprocessing exchage table
exchange_rates = standardize_col_names(exchange_rates)
exchange_rates = exchange_rates.apply(lambda row : format_date_column(row,date_col_name='date'),axis=1)



In [76]:
#create sql table and dumping data into it

import mysql.connector

con = mysql.connector.connect(
        host='localhost',
        user='root',
        password='password',
        database = 'global_electronics')
cursor = con.cursor()

In [9]:
# create database 
cursor.execute('show databases')
for val in cursor:
    print(val)

cursor.execute('create database if not exists global_electronics')


('global_electronics',)
('information_schema',)
('mysql',)
('performance_schema',)
('red_bus_scrape',)
('sys',)


In [10]:
cursor.execute('show tables')
tables = []
for val in cursor:
    tables.append(val[0])
    print(val)

('customer',)
('exchange_rate',)
('product',)
('product_category',)
('sale',)
('store',)


In [ ]:
cursor.execute('''
 CREATE TABLE if not exists customer(
         customer_key int,
         gender varchar(10) NOT NULL,
         name varchar(50) NOT NULL,
         city varchar(50) NOT NULL,
         state_code varchar(10) NOT NULL,
         state varchar(50) NOT NULL,
         zip_code varchar(20) NOT NULL,
         country varchar(30) NOT NULL,
         continent varchar(30) NOT NULL,
         birthday date,
        PRIMARY KEY(customer_key))
''')


cursor.execute('''
                CREATE TABLE if not exists product_category(
                        category_id int,
                        sub_category varchar(50) NOT NULL,
                        category varchar(40) NOT NULL,
                        PRIMARY KEY(category_id),
               )
               ''')

cursor.execute(
    '''
        CREATE TABLE IF NOT EXISTS product(
                product_key int,
                name varchar(100) NOT NULL,
                brand varchar(30) NOT NULL,
                color varchar(20) NOT NULL,
                unit_cost decimal(7,2) NOT NULL,
                unit_price_usd decimal(5,2) NOT NULL,
                category_id int NOT NULL,
                PRIMARY KEY(product_key),
                FOREIGN KEY(category_id) REFERENCES product_category(category_id),
        )
    '''
)

cursor.execute(
    '''
    CREATE TABLE IF NOT EXISTS store(
        store_key int,
        country varchar(20) NOT NULL,
        state varchar(50) NOT NULL,
        square_meters Decimal(10,2) NOT NULL,
        open_date date NOT NULL,
        PRIMARY KEY(store_key),
    )
    ''')



cursor.execute(
    '''
    CREATE TABLE IF NOT EXISTS sale(
        order_name int,
        line_item int NOT NULL,
        order_date date NOT NULL,
        delivery_date date, 
        customer_key int NOT NULL,
        store_key int NOT NULL,
        product_key int NOT NULL,
        quantity int NOT NULL,
        currency_code varchar(6) NOT NULL,
        PRIMARY KEY(order_name, line_item),
        FOREIGN KEY(customer_key) REFERENCES customer(customer_key),
        FOREIGN KEY(store_key) REFERENCES store(store_key),
        FOREIGN KEY(product_key) REFERENCES product(product_key)
    )
    '''
)

cursor.execute(
    '''
    CREATE TABLE IF NOT EXISTS exchange_rate(
    date date,
    currency varchar(7),
    exchange decimal(7,4))
    
    '''
)


In [105]:
for val in sales['customerkey']:
    if val not in customers['customerkey']:
        print(val)

265598
1269051
1269051
266019
266019
266019
1107461
1107461
844003
2035771
2035771
759705
759705
759705
254540
370077
1984985
1984985
1984985
1984985
1977527
1977527
1977527
1187306
738549
738549
1982762
1438050
980164
1853699
1853699
758280
758280
758280
758280
758280
758280
758280
693285
1730985
1730985
763246
1512558
1512558
1512558
1512558
1512558
1322959
1322959
507981
209990
209990
815458
815458
1839627
911025
1633409
1633409
1633409
1612400
1301565
1301565
1301565
793573
793573
793573
799366
799366
243917
243917
243917
942807
942807
942807
942807
129318
129318
1713253
1713253
1736623
1736623
1736623
1836599
1836599
1482586
1482586
1482586
607356
607356
1260533
1260533
1843882
1843882
909014
909014
909014
909014
892838
1670789
1143828
1143828
269337
269337
258316
1389462
1389462
1866637
734605
571622
571622
571622
1664824
1696695
1696695
1696695
1696695
2032025
2032025
2032025
1884908
1884908
1884908
2092033
2092033
1473090
584095
584095
661636
623659
623659
183264
1393738
139373

In [88]:
cursor.execute('select count(*) from exchange_rate')
for val in cursor:
    print(val)

(11215,)


In [ ]:
cursor.executemany('''
    INSERT INTO sale (order_name, line_item, order_date, delivery_date, customer_key, store_key, product_key, quantity, currency_code)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
''', sales.values.tolist())

In [101]:
#dumping values in the createad table
# Insert data into customer table
# cursor.executemany('''
#     INSERT INTO customer (customer_key, gender, name, city, state_code, state, zip_code, country, continent, birthday)
#     VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
# ''', customers.values.tolist())

# Insert data into product_category table
# cursor.executemany('''
#     INSERT INTO product_category (category_id, sub_category, category)
#     VALUES (%s, %s, %s)
# ''', product_category.values.tolist())

# Insert data into product table
# cursor.executemany('''
#     INSERT INTO product (product_key, name, brand, color, unit_cost, unit_price_usd, category_id)
#     VALUES (%s, %s, %s, %s, %s, %s, %s)
# ''', products.values.tolist())

# Insert data into store table
# cursor.executemany('''
#     INSERT INTO store (store_key, country, state, square_meters, open_date)
#     VALUES (%s, %s, %s, %s, %s)
# ''', stores.values.tolist())

# Insert data into sale table
cursor.executemany('''
    INSERT INTO sale (order_name, line_item, order_date, delivery_date, customer_key, store_key, product_key, quantity, currency_code)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
''', sales.values.tolist())

# Insert data into exchange_rate table
# cursor.executemany('''
#     INSERT INTO exchange_rate (date, currency, exchange)
#     VALUES (%s, %s, %s)
# ''', exchange_rates.values.tolist())



IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`global_electronics`.`sale`, CONSTRAINT `sale_ibfk_1` FOREIGN KEY (`customer_key`) REFERENCES `customer` (`customer_key`))

#### customers table
- Nopoli's country code is missing and I found that Napoli is also called Naples whose state code is NA.
- convert birthday into Date format for sql.
- normalize the column names.
- try separating state and country into separate table

#### products table
- split the product table further ie subcategory and category.
- normalize the column names.

#### sales table
- delivery date has missing values but we could not fill that.
- normalize the column names.
- create a primary key for sql
- format date columns


#### stores table
- convert the open date to dateformat for sql
- normalize the column names
- handle one NaN value for Online

#### xchange table
- normalize the column names
- convert the 'date' column to date format
